# Description of sample

We'll create a small example of an AI application that responds to users' queries based on a MySQL table of product reviews. 

At the end of the example, the solution will look something like this:

```
[User search]: Light weight gym bag
[AI Response]: After searching through our product database, I recommend <product ID> because... 
```

Behind the scenes, we take the following steps:
* Prep work: Set up a sample table in a Azure MySQL Flexible Server DB and upload data to it
* Step 1 - Set up data source connection in Azure AI Search 
* Step 2 - Set up automatic chunking + vectorization + indexing 
  * Set up an index in Azure AI Search to store the data we need, including vectorized versions of the text reviews.
  * Set up an indexer in Azure AI Search to pull data into the index 
  * Add a skillset to automatically chunk and vectorize the data using an Azure OpenAI Embedding service
* Step 3 - Use vector search from a sample application 
* Step 4 - Generate a GPT response to the user 
  * Create an Azure OpenAI Chat Completion service to utilize the Azure AI Search as datasource.
  * Use the Azure OpenAI Chat Completion service to respond to the user's query 

Copyright (c) Microsoft Corporation.
Licensed under the MIT license.

## Pre-requisites

You will need:
* An existing Azure Database for MySQL Flexible Server with server name, DB name, username, and password copied into `example.env`
  * The user must have permission to create a new table
  * You must whitelist your IP to access your Azure Database for MySQL flexible server by opening the MySQL server resource in the Azure portal, navigating to Security / Networking, and adding your IP.
* An OpenAI resource with the endpoint and key copied into `example.env`
* An Azure AI Search resource with the endpoint and key copied into `example.env`
* The Python packages listed in `requirements.txt` (can be installed using `pip`)


# Load environment variables and keys

In [ ]:
from dotenv import dotenv_values
# specify the name of the .env file name 
env_name = "./src/example.env"
config = dotenv_values(env_name)

In [ ]:
# Load Azure database for MySQL flexible server connection details
server = config["server"]
database = config["database"]
username = config["username"]
password = config["password"]

In [ ]:
# Load Open AI deployment details
import openai
openai.api_type = config["openai_api_type"]
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_base']
openai.api_version = config['openai_api_version'] 
openai_deployment_embedding = config["openai_deployment_embedding"]
openai_deployment_completion = config["openai_deployment_completion"]
EMBEDDING_LENGTH = 1536

In [ ]:
# Load Cognitive Search service details
azureai_search_key = config["azureai_search_api_key"]
service_endpoint = config["azureai_search_endpoint"]
index_name = config["azureai_search_index_name"] # Desired name of index -- does not need to exist already


# Upload data to MySQL DB

## Create table

### Connect to database

For simplicity, we set `autocommit=True` in the mysql connector parameters, which allows us to execute `ALTER` statements. 

If a timeout error occurs, retry the cell.

In [ ]:
import mysql.connector

# Establish a connection to the Azure MySQL database
conn = mysql.connector.connect(host=server, user=username, password=password, database=database, autocommit=True, ssl_disabled=False, ssl_verify_cert=False, ssl_verify_identity=False)
cursor = conn.cursor()

### Create a table in the database

We will create a new table "foodreview" and upload the data from a csv file.

In [ ]:
table_name = "productreviews" 

# Drop previous table of same name if one exists
cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
print("Finished dropping table (if existed)")

# Create a table
cursor.execute(f"""
               CREATE TABLE {table_name} 
               (Id integer NOT NULL, 
               ProductId text, 
               ProductName text, 
               ProductImage text, 
               ProductUrl text, 
               ReviewId text, 
               Summary text, 
               Text text,
               ReviewerNickName text,
               TextConcat text,
               Created DATETIME DEFAULT CURRENT_TIMESTAMP,
               LastUpdated DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
               Deleted integer DEFAULT 0,
               PRIMARY KEY (Id)
               );
               """)
print("Finished creating table")


server_change_detection_column_name = "LastUpdated"
server_soft_delete_column_name = "Deleted"
server_soft_delete_marker_value = "1"

## Upload data

### Load data from CSV

The data contains a few product reviews, with related info.

In [ ]:
## Load Data
import numpy as np
import pandas as pd
df_all = pd.read_csv('./DataSet/magento_reviews.csv')

df_all.head(3)

### Manipulate data

For our example, we will combine the user's summary with the user's review text.

In [ ]:
df_all["TextConcat"] = df_all.apply(lambda row: f"Summary: {row['Summary']} | Review: {row['Text']}",
                                    axis = 1)

df_all.head(3)

### Upload to DB

In [ ]:
# Split the dataframe into batches
batch_size = 30
batches = [df_all[i:i + batch_size] for i in range(0, len(df_all), batch_size)]

#Iterate over each batch and insert the data into the database
for batch in batches:
    # Convert the batch dataframe to a list of tuples for bulk insertion
    rows = [tuple(row) for row in batch.itertuples(index=False)]
    
    # Define the SQL query for bulk insertion
    query = f"INSERT INTO {table_name} (Id, ProductId, ProductName, ProductImage, ProductUrl, ReviewId, Summary, Text, ReviewerNickName, TextConcat) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    cursor.executemany(query, rows)

## Example query

This checks that the data was uploaded correctly. We should have 99 rows at this point.

In [ ]:
# Execute the SELECT statement
try:
    cursor.execute(f"SELECT count(Id) FROM {table_name};")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except Exception as e:
    print(f"Error executing SELECT statement: {e}")

# Step 1 - Set up data source connection in Azure AI Search

## Import needed Azure AI functions

In [ ]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryLanguage,
    QueryType,
    VectorizableTextQuery,
    VectorFilterMode,    
)
from azure.search.documents.indexes.models import (   
    AzureOpenAIParameters,  
    AzureOpenAIVectorizer,
    HnswAlgorithmConfiguration,
    HnswParameters,
    SemanticPrioritizedFields,   
    SearchField,  
    SearchFieldDataType,  
    SearchIndex,   
    SearchIndexerDataContainer,  
    SemanticConfiguration,  
    SemanticField,
    SemanticSearch,
    VectorSearch,  
    VectorSearchAlgorithmMetric,  
    VectorSearchProfile,
    SplitSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    AzureOpenAIEmbeddingSkill,
    SearchIndexerIndexProjections,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerSkillset,
    HighWaterMarkChangeDetectionPolicy,
    SoftDeleteColumnDeletionDetectionPolicy,
    SearchIndexerDataSourceConnection,
    SearchIndexer
)  

## Create data source connection

This step creates a connection that will be used to pull data from our MySQL table.

Documentation can be found [here.](https://learn.microsoft.com/en-us/azure/search/search-howto-index-mysql)

In [ ]:
ds_conn_str = f'Server={server}; Port=3306; Database={database}; Uid={username}; Pwd={password}; SslMode=Preferred;'

azureai_search_credential = AzureKeyCredential(azureai_search_key)
ds_client = SearchIndexerClient(service_endpoint, azureai_search_credential)
container = SearchIndexerDataContainer(name=table_name)

change_detection_policy = HighWaterMarkChangeDetectionPolicy(high_water_mark_column_name=server_change_detection_column_name)

soft_delete_detection_policy = SoftDeleteColumnDeletionDetectionPolicy(
    soft_delete_column_name=server_soft_delete_column_name,
    soft_delete_marker_value=server_soft_delete_marker_value)

data_source_connection = SearchIndexerDataSourceConnection(
    name=f"{index_name}-mysql-connection",
    type="mysql",
    connection_string=ds_conn_str,
    container=container,
    data_change_detection_policy=change_detection_policy,
    data_deletion_detection_policy=soft_delete_detection_policy
)

data_source = ds_client.create_or_update_data_source_connection(data_source_connection)

print(f"Data source '{data_source.name}' created or updated")

# Step 2 - Set up automatic chunking, vectorization and indexing

## Create index

The plan is:
1. Take the combined text (summary + review text) from each product review
2. Split the combined text into chunks
3. Embed each chunk as a vector
4. (Later) search for the most relevant chunk based on the incoming query. 

To enable this, the search index will store all of the following data, for each chunk of text:
* Id of chunk
* Chunk text
* Vector version of chunk text
* Id of parent row
* Product Id from parent row
* Review text from parent row
* Summary text from parent row
* Score from parent row

All of these values will be stored in SearchFields specified in the code below.

In this step we also configure the search algorithm(s), and the vectorizer that will automatically vectorize the incoming query.

Documentation about creating indexes can be found [here.](https://learn.microsoft.com/en-us/azure/search/search-how-to-create-search-index?tabs=index-other-sdks)

In [ ]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=azureai_search_credential)

fields = [
    # Properties of individual chunk
    SearchField(name="Id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=EMBEDDING_LENGTH, vector_search_profile_name="my-vector-search-profile"),
    # Properties of original row in DB that the chunk belonged to
    SearchField(name="parent_id", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_product_id", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_product_name", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_product_url", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_text", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
    SearchField(name="parent_summary", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="my-hnsw-config",
            parameters = HnswParameters( 
                m=4,  
                ef_construction=400,  
                ef_search=500,  
                metric=VectorSearchAlgorithmMetric.COSINE,  
            ),
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="my-vector-search-profile",
            algorithm_configuration_name="my-hnsw-config",
            vectorizer="my-openai"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="my-openai",
            kind="azureOpenAI",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri=openai.api_base,
                deployment_id=openai_deployment_embedding,
                api_key=openai.api_key
            )
        )  
    ]  
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        content_fields=[SemanticField(field_name="chunk")]
    )
)

# Create the semantic search with the configuration  
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f'{result.name} created')

## Create skillset

We use two pre-built skills:
1. The Split Skill takes the concatenated text and divides it into chunks (to stay within the token limits for the OpenAI embedding service).
2. The Azure Open AI Embedding Skill takes the outputs of the Split Skill and vectorizes them individually.

Afterwards, we apply an Index Projector to make it so that our final index has one item for every chunk of text (rather than one item for every original row in the DB).

We recommend the following resources to learn more about the process and how one can adapt it to different applications:
* [Overview of indexers](https://learn.microsoft.com/en-us/azure/search/search-indexer-overview)
* [Skill context and input annotation language](https://learn.microsoft.com/en-us/azure/search/cognitive-search-skill-annotation-language)
* [Reference inputs and outputs in skillsets](https://learn.microsoft.com/en-us/azure/search/cognitive-search-concept-annotations-syntax)

In [ ]:
# Create a skillset  
skillset_name = f"{index_name}-skillset"

split_skill = SplitSkill(  
    description="Split skill to chunk documents",  
    text_split_mode="pages",  
    context="/document",  
    maximum_page_length=300,  
    page_overlap_length=20,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/TextConcat"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="textItems", target_name="pages")  
    ]  
)

embedding_skill = AzureOpenAIEmbeddingSkill(  
    description="Skill to generate embeddings via Azure OpenAI",  
    context="/document/pages/*",  
    resource_uri=openai.api_base,  
    deployment_id=openai_deployment_embedding,  
    api_key=openai.api_key,  
    inputs=[  
        InputFieldMappingEntry(name="text", source="/document/pages/*"),  
    ],  
    outputs=[  
        OutputFieldMappingEntry(name="embedding", target_name="vector")  
    ]  
)  

index_projections = SearchIndexerIndexProjections(  
    selectors=[  
        SearchIndexerIndexProjectionSelector(  
            target_index_name=index_name,  
            parent_key_field_name="parent_id", # Note: this populates the "parent_id" search field
            source_context="/document/pages/*",  
            mappings=[  
                InputFieldMappingEntry(name="chunk", source="/document/pages/*"),
                InputFieldMappingEntry(name="vector", source="/document/pages/*/vector"),
                InputFieldMappingEntry(name="parent_product_id", source="/document/ProductId"),
                InputFieldMappingEntry(name="parent_product_name", source="/document/ProductName"),
                InputFieldMappingEntry(name="parent_product_url", source="/document/ProductUrl"),
                InputFieldMappingEntry(name="parent_text", source="/document/Text"),
                InputFieldMappingEntry(name="parent_summary", source="/document/Summary")
            ],  
        ),  
    ],
)  

skillset = SearchIndexerSkillset(  
    name=skillset_name,  
    description="Skillset to chunk documents and generating embeddings",  
    skills=[split_skill, embedding_skill],
    index_projections=index_projections  
)
  
client = SearchIndexerClient(service_endpoint, azureai_search_credential)  
client.create_or_update_skillset(skillset)  
print(f' {skillset.name} created')

## Create indexer

In [ ]:
# Create an indexer  
indexer_name = f"{index_name}-indexer"  

indexer = SearchIndexer(  
    name=indexer_name,  
    description="Indexer to chunk documents and generate embeddings",  
    skillset_name=skillset_name,  
    target_index_name=index_name,  
    data_source_name=data_source.name
)  
  
indexer_client = SearchIndexerClient(service_endpoint, azureai_search_credential)
indexer_result = indexer_client.create_or_update_indexer(indexer)  

# Run the indexer  
indexer_client.run_indexer(indexer_name)
print(f' {indexer_name} created')

In [ ]:
# Get the status of the indexer  
indexer_status = indexer_client.get_indexer_status(indexer_name)
print(f"Indexer status: {indexer_status.status}")

In [ ]:
# Allow some time for the indexer to process the data
import time
time.sleep(30)

# Step 3 - Use vector search from a sample application

## Perform queries

In [ ]:
user_query = "Light weight gym bag"

In the following output, we find the top 3 chunks that are most relevant to the user's query.

Feel free to retry the following cell in case of an empty response or a 429 error. An empty response probably indicates that the chunking/embedding process has not finished yet. A 429 error means there have been too many requests to the OpenAI embedding service and should go away on retrying.

In [ ]:
search_client = SearchClient(service_endpoint, index_name, credential=azureai_search_credential)
vector_query = VectorizableTextQuery(text=user_query, k_nearest_neighbors=3, fields="vector", exhaustive=True)
  
results = search_client.search(
    search_text=user_query,  
    vector_queries= [vector_query],
    select=["Id", "parent_id", "chunk", "parent_product_id", "parent_product_name", "parent_product_url", "parent_text", "parent_summary"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='my-semantic-config',
    top=5
)

for result in results:
    print(f"Search score: {result['@search.score']}")
    print(f"Search re-ranker score: {result['@search.reranker_score']}")
    print(f"Parent Id: {result['parent_id']} | Chunk id: {result['Id']}")
    print(f"Product Name: {result['parent_product_name']}")
    print(f"Product Url: {result['parent_product_url']}")
    print(f"Text chunk: {result['chunk']}") 
    print(f"Review summary: {result['parent_summary']}")
    print(f"Review text: {result['parent_text']}")
    print("-----")


# Step 4 - Generate a GPT Response to the user

## Prompt creation

In [ ]:
# create a prompt for AI chat completion 
system_prompt = """
    You are an AI assistant that recommends product to people based on the product reviews data matching their query. 
    Your answer should summarize the review text, include the product ID, include the parent id as review id, and mention the overall sentiment of the review.
    """

## Call to OpenAI

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    base_url=f"{openai.api_base}/openai/deployments/{openai_deployment_completion}/extensions",
    api_key=openai.api_key,
    api_version=openai.api_version,
)

message_text = [{"role": "user", "content": user_query}]

completion = client.chat.completions.create(
    messages=message_text,
    model=openai_deployment_completion,
    extra_body={
        "dataSources":[
            {
                "type": "AzureCognitiveSearch",
                "parameters": {
                    "endpoint": service_endpoint,
                    "indexName": index_name,
                    "semanticConfiguration": "my-semantic-config",
                    "queryType": "vectorSemanticHybrid",
                    "inScope": True,
                    "roleInformation": system_prompt,
                    "strictness": 3,
                    "topNDocuments": 5,
                    "key": azureai_search_key,
                    "embeddingDeploymentName": openai_deployment_embedding
                }
            }
        ],
    },
    n=3,
    temperature=1,
    top_p=1,
    max_tokens=1024
)

print(completion.model_dump_json(indent=2))